In [ ]:
import pandas as pd
# lplndf = pd.read_json("logicalplan_q1.json")
lplndf = pd.read_json("physicalplan_q1ext.json")
lplndf

In [50]:
import pandas as pd
import dask.dataframe as dd
import re
from timeit import default_timer as timer

tables_sizes = {}
table_division = {}

col_names_lineitem = ['l_orderkey','l_partkey','l_suppkey','l_linenumber','l_quantity','l_extendedprice','l_discount','l_tax','l_returnflag','l_linestatus','l_shipdate','l_commitdate','l_receiptdate','l_shipinstruct','l_shipmode', 'l_comment']
col_names_orders = ['o_orderkey','o_custkey','o_orderstatus','o_totalprice','o_orderdate','o_orderpriority','o_clerk','o_shippriority','o_comment']
col_names_customer = ['c_custkey','c_name','c_address','c_nationkey','c_phone','c_acctbal','c_mktsegment','c_comment']     

dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')

def set_table_size(df_string, partition_size):
        tables_sizes[df_string] = partition_size
        if not df_string in table_division:
            table_division[df_string] = {}

def get_table_size(df_string):
        return tables_sizes[df_string]

def good_to_set_index(df_string):
        if df_string in tables_sizes:
            if tables_sizes[df_string] > 3:
                return True
            return True

def set_table_division(df_string,df_col,division):
        table_division[df_string][df_col] = division


def get_table_division(df_string,df_col):
        if df_col in table_division[df_string]:
            return table_division[df_string][df_col]
        return None

def print_table_sizes():
        print(tables_sizes)

def print_table_division():
        print(table_division)
        

class DaskPlan:
    
    column_mappings = {}
    index_count = {}
    set_indexes = {}
    raco_indexes = {}
    one_order_by = False
    one_limit = False
    scale_factor='1'
    
    def parse_date(self,line):
            line = re.sub(r"date_", "", line)
            return 'pd.to_datetime("' + line +'")'

    def parse_interval(self,line):
            line = re.sub(r"interval\s", "", line)
            interval_list = line.split(" ")
            int(interval_list[0])
            if interval_list[1] == 'DAY':
                arg1 = 'days'
            elif interval_list[1] == 'MONTH':
                arg1 = 'months'
            elif interval_list[1] == 'YEAR':
                arg1 = 'years'
            return 'pd.DateOffset(' + arg1 + '=' + interval_list[0] + ')'

    def add_columns_index(self,df,df_string):
            self.column_mappings[df_string] = df.columns

    def add_dataframe_index_count(self,df_string,count,indexes):
            self.index_count[df_string] = count
            self.set_indexes[df_string] = indexes

    def get_dataframe_index(self,df_string):
            if df_string in self.set_indexes:
                return self.set_indexes[df_string]
            return []

    def get_dataframe_index_count(self,df_string):
            if df_string not in self.index_count:
                return 0
            return self.index_count[df_string]
    
    def run_q1(self):
        start = timer()
        
        lineitem = pd.read_csv('lineitem.csv',delimiter='|',names=col_names_lineitem, usecols=['l_orderkey', 'l_extendedprice', 'l_discount'], parse_dates=[], date_parser=dateparse)
        self.lineitem = lineitem
        # set_table_size('lineitem',self.lineitem.npartitions)
        set_table_size('lineitem',12)
        orders  = pd.read_csv('orders.csv',delimiter='|',names=col_names_orders, usecols=['o_orderkey', 'o_orderdate'], parse_dates=['o_orderdate'], date_parser=dateparse)
        self.orders = orders
        # set_table_size('orders',self.orders.npartitions)
        set_table_size('orders',3)
        # print self.orders.head(5)
        start1 = timer()
        self.add_columns_index(lineitem,"lineitem")
        self.add_columns_index(orders,"orders")
        # print 'here: ', self.column_mappings["orders"][1]
        
        orders=orders[orders[self.column_mappings["orders"][1]]>=pd.to_datetime("1995-01-01")]
        orders=orders.rename(columns={self.column_mappings["orders"][0]:"o_orderkey"})
        self.add_columns_index(orders,"orders")
        orders=orders.loc[:,["o_orderkey"]]
        self.add_columns_index(orders,"orders")
        divisions = get_table_division("lineitem",self.column_mappings["lineitem"][0])
        # lineitem=lineitem.set_index(self.column_mappings["lineitem"][0], divisions=divisions)
        lineitem=lineitem.set_index(self.column_mappings["lineitem"][0])
        # set_table_division("lineitem",self.column_mappings["lineitem"][0],lineitem.divisions)
        indexed_col2 =self.column_mappings["lineitem"][0]
        merged_table_lineitem=orders.merge(lineitem,left_on=self.column_mappings["orders"][0],right_index=True)
        len_col1 =len(self.column_mappings["orders"])
        
        self.add_columns_index(merged_table_lineitem,"merged_table_lineitem")
        merged_table_lineitem=merged_table_lineitem.loc[:,[self.column_mappings["merged_table_lineitem"][1+len_col1-1],self.column_mappings["merged_table_lineitem"][2+len_col1-1]]]
        merged_table_lineitem=merged_table_lineitem.reset_index()
        merged_table_lineitem=merged_table_lineitem.rename(columns={"index":indexed_col2})
        self.add_columns_index(merged_table_lineitem,"merged_table_lineitem")
        merged_table_lineitem=merged_table_lineitem.loc[:,[self.column_mappings["merged_table_lineitem"][0],self.column_mappings["merged_table_lineitem"][1],self.column_mappings["merged_table_lineitem"][2],]]
        self.add_columns_index(merged_table_lineitem,"merged_table_lineitem")
        merged_table_lineitem=merged_table_lineitem.rename(columns={self.column_mappings["merged_table_lineitem"][0]:"l_orderkey"})
        self.add_columns_index(merged_table_lineitem,"merged_table_lineitem")
        merged_table_lineitem["_COLUMN1_"]=merged_table_lineitem[self.column_mappings["merged_table_lineitem"][1]]*(1-merged_table_lineitem[self.column_mappings["merged_table_lineitem"][2]])
        merged_table_lineitem=merged_table_lineitem.loc[:,["l_orderkey","_COLUMN1_"]]
        self.add_columns_index(merged_table_lineitem,"merged_table_lineitem")
        
        gb=merged_table_lineitem.groupby([self.column_mappings["merged_table_lineitem"][0]])
        merged_table_lineitem=gb.agg({self.column_mappings["merged_table_lineitem"][1]:"sum"})
        merged_table_lineitem=merged_table_lineitem.reset_index()
        
        self.add_columns_index(merged_table_lineitem,"merged_table_lineitem")
        merged_table_lineitem=merged_table_lineitem.rename(columns={self.column_mappings["merged_table_lineitem"][0]:"l_orderkey",self.column_mappings["merged_table_lineitem"][1]:"revenue"})
        self.add_columns_index(merged_table_lineitem,"merged_table_lineitem")
        merged_table_lineitem=merged_table_lineitem.loc[:,["l_orderkey","revenue"]]
        self.add_columns_index(merged_table_lineitem,"merged_table_lineitem")
        
        merged_table_lineitem=merged_table_lineitem.nlargest(5,columns=["revenue"])
        
        end = timer()
        print "Total time taken: ", str(end-start) + "\n"
        print "Time taken (except file loadng): ", str(end-start1) + "\n"
        
        return merged_table_lineitem
    
    
    def run_q5(self):
        self.add_columns_index(orders,"orders")
        orders=orders[(orders[self.column_mappings["orders"][2]]>=pd.to_datetime("1995-01-01"))&(orders[self.column_mappings["orders"][2]]<(pd.to_datetime("1995-01-01")+pd.DateOffset(years=1)))]
        self.add_columns_index(customer,"customer")
        self.add_columns_index(lineitem,"lineitem")
        divisions = get_table_division("orders",self.column_mappings["orders"][1])
        orders=orders.set_index(self.column_mappings["orders"][1], divisions=divisions)
        set_table_division("orders",self.column_mappings["orders"][1],orders.divisions)
        indexed_col2 =self.column_mappings["orders"][1]
        merged_table_orders=customer.merge(orders,left_on=self.column_mappings["customer"][0],right_index=True)
        len_col1 =len(self.column_mappings["customer"])
        self.add_columns_index(merged_table_orders,"merged_table_orders")
        merged_table_orders=merged_table_orders.loc[:,[self.column_mappings["merged_table_orders"][0],self.column_mappings["merged_table_orders"][1],self.column_mappings["merged_table_orders"][0+len_col1],self.column_mappings["merged_table_orders"][2+len_col1-1]]]
        merged_table_orders=merged_table_orders.reset_index()
        merged_table_orders=merged_table_orders.rename(columns={"index":indexed_col2})
        self.add_columns_index(merged_table_orders,"merged_table_orders")
        merged_table_orders=merged_table_orders.loc[:,[self.column_mappings["merged_table_orders"][1],self.column_mappings["merged_table_orders"][2],self.column_mappings["merged_table_orders"][3],self.column_mappings["merged_table_orders"][0],self.column_mappings["merged_table_orders"][4],]]
        self.add_columns_index(merged_table_orders,"merged_table_orders")
        self.add_columns_index(supplier,"supplier")
        divisions = get_table_division("lineitem",self.column_mappings["lineitem"][0])
        lineitem=lineitem.set_index(self.column_mappings["lineitem"][0], divisions=divisions)
        set_table_division("lineitem",self.column_mappings["lineitem"][0],lineitem.divisions)
        indexed_col2 =self.column_mappings["lineitem"][0]
        merged_table_lineitem=merged_table_orders.merge(lineitem,left_on=self.column_mappings["merged_table_orders"][2],right_index=True)
        len_col1 =len(self.column_mappings["merged_table_orders"])
        self.add_columns_index(merged_table_lineitem,"merged_table_lineitem")
        merged_table_lineitem=merged_table_lineitem.loc[:,[self.column_mappings["merged_table_lineitem"][0],self.column_mappings["merged_table_lineitem"][1],self.column_mappings["merged_table_lineitem"][2],self.column_mappings["merged_table_lineitem"][3],self.column_mappings["merged_table_lineitem"][4],self.column_mappings["merged_table_lineitem"][1+len_col1-1],self.column_mappings["merged_table_lineitem"][2+len_col1-1],self.column_mappings["merged_table_lineitem"][3+len_col1-1]]]
        merged_table_lineitem=merged_table_lineitem.reset_index()
        merged_table_lineitem=merged_table_lineitem.rename(columns={"index":indexed_col2})
        self.add_columns_index(merged_table_lineitem,"merged_table_lineitem")
        merged_table_lineitem=merged_table_lineitem.loc[:,[self.column_mappings["merged_table_lineitem"][1],self.column_mappings["merged_table_lineitem"][2],self.column_mappings["merged_table_lineitem"][3],self.column_mappings["merged_table_lineitem"][4],self.column_mappings["merged_table_lineitem"][5],self.column_mappings["merged_table_lineitem"][0],self.column_mappings["merged_table_lineitem"][6],self.column_mappings["merged_table_lineitem"][7],self.column_mappings["merged_table_lineitem"][8],]]
        self.add_columns_index(merged_table_lineitem,"merged_table_lineitem")
        self.add_columns_index(nation,"nation")
        merged_table_supplier=merged_table_lineitem.merge(supplier,left_on=[self.column_mappings["merged_table_lineitem"][1],self.column_mappings["merged_table_lineitem"][6]],right_on=[self.column_mappings["supplier"][1],self.column_mappings["supplier"][0]])
        len_col1 =len(self.column_mappings["merged_table_lineitem"])
        self.add_columns_index(merged_table_supplier,"merged_table_supplier")
        merged_table_supplier=merged_table_supplier.loc[:,[self.column_mappings["merged_table_supplier"][0],self.column_mappings["merged_table_supplier"][1],self.column_mappings["merged_table_supplier"][2],self.column_mappings["merged_table_supplier"][3],self.column_mappings["merged_table_supplier"][4],self.column_mappings["merged_table_supplier"][5],self.column_mappings["merged_table_supplier"][6],self.column_mappings["merged_table_supplier"][7],self.column_mappings["merged_table_supplier"][8],self.column_mappings["merged_table_supplier"][0+len_col1],self.column_mappings["merged_table_supplier"][1+len_col1]]]
        self.add_columns_index(merged_table_supplier,"merged_table_supplier")
        self.add_columns_index(region,"region")
        region=region[region[self.column_mappings["region"][1]]=="AMERICA"]
        region=region.rename(columns={self.column_mappings["region"][0]:"r_regionkey"})
        self.add_columns_index(region,"region")
        region=region.loc[:,["r_regionkey"]]
        self.add_columns_index(region,"region")
        divisions = get_table_division("nation",self.column_mappings["nation"][0])
        nation=nation.set_index(self.column_mappings["nation"][0], divisions=divisions)
        set_table_division("nation",self.column_mappings["nation"][0],nation.divisions)
        indexed_col2 =self.column_mappings["nation"][0]
        merged_table_nation=merged_table_supplier.merge(nation,left_on=self.column_mappings["merged_table_supplier"][10],right_index=True)
        len_col1 =len(self.column_mappings["merged_table_supplier"])
        self.add_columns_index(merged_table_nation,"merged_table_nation")
        merged_table_nation=merged_table_nation.loc[:,[self.column_mappings["merged_table_nation"][7],self.column_mappings["merged_table_nation"][8],self.column_mappings["merged_table_nation"][1+len_col1-1],self.column_mappings["merged_table_nation"][2+len_col1-1]]]
        merged_table_nation=merged_table_nation.reset_index(drop=True)
        self.add_columns_index(merged_table_nation,"merged_table_nation")
        divisions = get_table_division("region",self.column_mappings["region"][0])
        region=region.set_index(self.column_mappings["region"][0], divisions=divisions)
        set_table_division("region",self.column_mappings["region"][0],region.divisions)
        indexed_col2 =self.column_mappings["region"][0]
        merged_table_region=merged_table_nation.merge(region,left_on=self.column_mappings["merged_table_nation"][3],right_index=True)
        len_col1 =len(self.column_mappings["merged_table_nation"])
        self.add_columns_index(merged_table_region,"merged_table_region")
        merged_table_region=merged_table_region.loc[:,[self.column_mappings["merged_table_region"][0],self.column_mappings["merged_table_region"][1],self.column_mappings["merged_table_region"][2]]]
        merged_table_region=merged_table_region.reset_index(drop=True)
        self.add_columns_index(merged_table_region,"merged_table_region")
        merged_table_region=merged_table_region.rename(columns={self.column_mappings["merged_table_region"][2]:"n_name"})
        self.add_columns_index(merged_table_region,"merged_table_region")
        merged_table_region["_COLUMN1_"]=merged_table_region[self.column_mappings["merged_table_region"][0]]*(1-merged_table_region[self.column_mappings["merged_table_region"][1]])
        merged_table_region=merged_table_region.loc[:,["n_name","_COLUMN1_"]]
        self.add_columns_index(merged_table_region,"merged_table_region")
        gb=merged_table_region.groupby([self.column_mappings["merged_table_region"][0]])
        merged_table_region=gb.agg({self.column_mappings["merged_table_region"][1]:"sum"})
        merged_table_region=merged_table_region.reset_index()
        self.add_columns_index(merged_table_region,"merged_table_region")
        merged_table_region=merged_table_region.rename(columns={self.column_mappings["merged_table_region"][0]:"n_name",self.column_mappings["merged_table_region"][1]:"revenue"})
        self.add_columns_index(merged_table_region,"merged_table_region")
        merged_table_region=merged_table_region.loc[:,["n_name","revenue"]]
        self.add_columns_index(merged_table_region,"merged_table_region")
        merged_table_region=merged_table_region.nlargest(1,columns=["revenue"])
    
    def run_q3(self):
                
        lineitem = pd.read_csv('lineitem.csv',delimiter='|',names=col_names_lineitem, usecols=['l_orderkey', 'l_extendedprice', 'l_discount', 'l_shipdate'], parse_dates=['l_shipdate'], date_parser=dateparse)
        self.lineitem = lineitem
        # set_table_size('lineitem',self.lineitem.npartitions)
        set_table_size('lineitem',12)
        
        orders  = pd.read_csv('orders.csv',delimiter='|',names=col_names_orders, usecols=['o_orderkey', 'o_custkey', 'o_orderdate', 'o_shippriority'], parse_dates=['o_orderdate'], date_parser=dateparse)
        self.orders = orders
        # set_table_size('orders',self.orders.npartitions)
        set_table_size('orders',3)
        
        customer  = pd.read_csv('customer.csv',delimiter='|',names=col_names_customer, usecols=['c_custkey', 'c_mktsegment'], parse_dates=[], date_parser=dateparse)
        self.customer = customer
        # set_table_size('orders',self.orders.npartitions)
        set_table_size('customer',3)
        
        self.add_columns_index(customer,"customer")
        customer=customer[customer[self.column_mappings["customer"][1]]=="HOUSEHOLD"]
        self.add_columns_index(orders,"orders")
        orders=orders[orders[self.column_mappings["orders"][2]]<pd.to_datetime("1995-03-21")]
        self.add_columns_index(lineitem,"lineitem")
        lineitem=lineitem[lineitem[self.column_mappings["lineitem"][3]]>pd.to_datetime("1995-03-21")]
        lineitem=lineitem.rename(columns={self.column_mappings["lineitem"][0]:"l_orderkey",self.column_mappings["lineitem"][1]:"l_extendedprice",self.column_mappings["lineitem"][2]:"l_discount"})
        self.add_columns_index(lineitem,"lineitem")
        lineitem=lineitem.loc[:,["l_orderkey","l_extendedprice","l_discount"]]
        self.add_columns_index(lineitem,"lineitem")
        divisions = get_table_division("customer",self.column_mappings["customer"][0])
        # customer=customer.set_index(self.column_mappings["customer"][0], divisions=divisions)
        # set_table_division("customer",self.column_mappings["customer"][0],customer.divisions)
        indexed_col2 =self.column_mappings["customer"][0]
        
        merged_table_customer=orders.merge(customer,left_on=self.column_mappings["orders"][1],right_index=True)
        len_col1 =len(self.column_mappings["orders"])
        self.add_columns_index(merged_table_customer,"merged_table_customer")
        merged_table_customer=merged_table_customer.loc[:,[self.column_mappings["merged_table_customer"][0],self.column_mappings["merged_table_customer"][2],self.column_mappings["merged_table_customer"][3]]]
        merged_table_customer=merged_table_customer.reset_index(drop=True)
        self.add_columns_index(merged_table_customer,"merged_table_customer")
        divisions = get_table_division("lineitem",self.column_mappings["lineitem"][0])
        # lineitem=lineitem.set_index(self.column_mappings["lineitem"][0], divisions=divisions)
        # set_table_division("lineitem",self.column_mappings["lineitem"][0],lineitem.divisions)
        indexed_col2 =self.column_mappings["lineitem"][0]
        
        merged_table_lineitem=merged_table_customer.merge(lineitem,left_on=self.column_mappings["merged_table_customer"][0],right_index=True)
        len_col1 =len(self.column_mappings["merged_table_customer"])
        self.add_columns_index(merged_table_lineitem,"merged_table_lineitem")
        
        print '1: ', merged_table_lineitem.head(5)
        
        # merged_table_lineitem=merged_table_lineitem.loc[:,[self.column_mappings["merged_table_lineitem"][1],self.column_mappings["merged_table_lineitem"][2],self.column_mappings["merged_table_lineitem"][1+len_col1-1],self.column_mappings["merged_table_lineitem"][2+len_col1-1]]]
        # merged_table_lineitem=merged_table_lineitem.reset_index()
        # merged_table_lineitem=merged_table_lineitem.rename(columns={"index":indexed_col2})
        
        print '2: ', merged_table_lineitem.head(5)
            
        self.add_columns_index(merged_table_lineitem,"merged_table_lineitem")
        merged_table_lineitem=merged_table_lineitem.loc[:,[self.column_mappings["merged_table_lineitem"][1],self.column_mappings["merged_table_lineitem"][2],self.column_mappings["merged_table_lineitem"][0],self.column_mappings["merged_table_lineitem"][3],self.column_mappings["merged_table_lineitem"][4],]]
        self.add_columns_index(merged_table_lineitem,"merged_table_lineitem")
        
        print '3: ', merged_table_lineitem.head(5) 
        
        merged_table_lineitem=merged_table_lineitem.rename(columns={self.column_mappings["merged_table_lineitem"][0]:"o_orderdate",self.column_mappings["merged_table_lineitem"][1]:"o_shippriority",self.column_mappings["merged_table_lineitem"][2]:"l_orderkey"})
        self.add_columns_index(merged_table_lineitem,"merged_table_lineitem")
        
        print '4: ', merged_table_lineitem.head(5) 
        print '4.5 :', self.column_mappings["merged_table_lineitem"][3]
        print '4.5.0 :', self.column_mappings["merged_table_lineitem"][4]
        print '4.5.1:', merged_table_lineitem[self.column_mappings["merged_table_lineitem"][3]].head(5)
        print '4.5.2:', merged_table_lineitem[self.column_mappings["merged_table_lineitem"][4]].head(5)
            
        merged_table_lineitem["_COLUMN3_"]=merged_table_lineitem[self.column_mappings["merged_table_lineitem"][3]]*(1-merged_table_lineitem[self.column_mappings["merged_table_lineitem"][4]])
        merged_table_lineitem=merged_table_lineitem.loc[:,["o_orderdate","o_shippriority","l_orderkey","_COLUMN3_"]]
        self.add_columns_index(merged_table_lineitem,"merged_table_lineitem")
        gb=merged_table_lineitem.groupby([self.column_mappings["merged_table_lineitem"][2],self.column_mappings["merged_table_lineitem"][0],self.column_mappings["merged_table_lineitem"][1]])
        merged_table_lineitem=gb.agg({self.column_mappings["merged_table_lineitem"][3]:"sum"})
        merged_table_lineitem=merged_table_lineitem.reset_index()
        self.add_columns_index(merged_table_lineitem,"merged_table_lineitem")
        merged_table_lineitem=merged_table_lineitem.rename(columns={self.column_mappings["merged_table_lineitem"][0]:"l_orderkey",self.column_mappings["merged_table_lineitem"][3]:"revenue",self.column_mappings["merged_table_lineitem"][1]:"o_orderdate",self.column_mappings["merged_table_lineitem"][2]:"o_shippriority"})
        self.add_columns_index(merged_table_lineitem,"merged_table_lineitem")
        merged_table_lineitem=merged_table_lineitem.loc[:,["l_orderkey","revenue","o_orderdate","o_shippriority"]]
        self.add_columns_index(merged_table_lineitem,"merged_table_lineitem")
        merged_table_lineitem=merged_table_lineitem.nlargest(10,columns=["revenue","o_orderdate"])
        return merged_table_lineitem

dp = DaskPlan()
df = dp.run_q1()
df

Total time taken:  3.28422212601

Time taken (except file loadng):  0.105713129044



,l_orderkey,revenue
50514,92685,443151.2042
57100,104951,440253.7446
34579,63414,438058.1785
3969,7293,434796.5388
43277,79416,431902.1246
